In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime
import pandas as pd
import csv

code = '225570'
company = 'nexon'
pages = 0

# 함수 : 마지막 페이지 찾기

def getLastPageNum(url):
    with urlopen(url) as u:
        html = BeautifulSoup(u, "lxml")
        last = html.find("td", class_ = "pgRR")
        s = str(last.a["href"]).split('=')
        lastpage = s[-1]
        return int(lastpage)
    
# 함수 : 페이지 정보 가져오기
def getPageValue(url , pages):
    df = pd.DataFrame()
    print("\n")
    for page in range(1, pages + 1): # 한 페이지씩 넘기면서 가져온다
        pg_url = f'{url}&page={page}'
        pValue = pd.read_html(pg_url, header = 0, encoding = 'cp949')[0]
        df = df.append(pValue)
        print(f'downloding.. {page:04d}/{pages:04d} : {company} ({code})' , end = '\r')
        print("\n")
        return df
    
# 함수 : 증시 정보 가져오기
def ReadStock(code, company, pages):
    try:
        url = (f"https://finance.naver.com/item/sise_day.naver?code={code}")

        # 입력값이 0이면 마지막 페이지 값을 가져온다.
        if pages == 0:
            pages = getLastPageNum(url)

        df = getPageValue(url, pages)   # 각 페이지마다의 정보를 가져온다.

        df = df.rename(columns = {'날짜' : 'date', '종가' : 'close' , '전일비' : 'diff'
                                  ,'시가' : 'open' , '고가' : 'high', '저가':'low'
                                  , '거래량' : 'volume'})
        df = df.dropna()    #결측치 제거
        # 연산을 위해 int 형으로 변경
        df[['close' , 'open' , 'high', 'low' , 'volume']] = df[['close' , 'open' , 'high', 'low' , 'volume']].astype(int)

        # date를 index에 넣고 date column을 삭제
        df.index = df['date']
        df.index = df.index.str.repace('.','')
        df = df.drop('date' , axis = 1)

        # string형인 index를 date형으로 변경
        df.index = pd.to_datetime(df.index, format= '%Y-%m-%d')
    except Exception as e:
        print('Exception :',str(e))
        return None
    return df

# 함수 : CSV 파일 저장
def saveToCSV(stokcValue): #CSV 파일 저장
    csv_f = csv.writer()
    csv_f.writerow(['date', 'open', 'high' ,'low', 'close', 'volume'])
    for i in range(len(stokcValue)):
        value = [stokcValue.index.values[i]
                 ,stokcValue.open.values[i]
                 ,stokcValue.high.values[i]
                 ,stokcValue.low.values[i]
                 ,stokcValue.close.values[i]
                 ,stokcValue.volume.values[i]]
        csv_f.writerow(value)





In [7]:
# 실행 
df = ReadStock(code,company, pages)

if df is None:
    print('데이터를 가져오지 못했습니다')
else:
    print(df)
    saveToCSV(df)


Exception : 'NoneType' object has no attribute 'a'
데이터를 가져오지 못했습니다
